In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import mlflow
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-05-03 17:26:27.736509: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 17:26:27.737361: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 17:26:27.741572: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-03 17:26:27.748774: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746293187.759509   31253 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746293187.76

In [2]:
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/03 17:26:30 WARN Utils: Your hostname, VamsiPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/03 17:26:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/03 17:26:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_df = pd.read_csv('train.csv')

In [4]:
train_df["Exited"].value_counts()

Exited
0    130113
1     34921
Name: count, dtype: int64

In [5]:
# Sample 34921 records where Exited == 0 and combine with all records where Exited == 1
sampled_df = pd.concat([
    train_df[train_df["Exited"] == 1],
    train_df[train_df["Exited"] == 0].sample(n=34921, random_state=42)
]).sample(frac=1, random_state=42)  # shuffle the resulting dataframe

print(sampled_df["Exited"].value_counts())

Exited
0    34921
1    34921
Name: count, dtype: int64


In [6]:
# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(sampled_df)

In [7]:
spark_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- CustomerId: long (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: long (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Tenure: long (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: long (nullable = true)
 |-- HasCrCard: double (nullable = true)
 |-- IsActiveMember: double (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: long (nullable = true)



In [8]:
# Drop identifier columns that are unlikely useful for prediction
sampled_df = sampled_df.drop(["id", "CustomerId", "Surname"], axis=1)

# One-hot encode categorical features: Geography and Gender
sampled_df = pd.get_dummies(sampled_df, columns=["Geography", "Gender"], drop_first=True)

# Separate features and target
X = sampled_df.drop("Exited", axis=1)
y = sampled_df["Exited"]

# Scale the feature set
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the deep learning model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

Epoch 1/10


/root/miniconda3/envs/ml_learning/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-03 17:26:50.187430: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1397/1397 ━━━━━━━━━━━━━━━━━━━━ 2s 959us/step - accuracy: 0.7324 - loss: 0.5232 - val_accuracy: 0.7987 - val_loss: 0.4357
Epoch 2/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 891us/step - accuracy: 0.7990 - loss: 0.4341 - val_accuracy: 0.8019 - val_loss: 0.4280
Epoch 3/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 874us/step - accuracy: 0.8006 - loss: 0.4280 - val_accuracy: 0.8033 - val_loss: 0.4241
Epoch 4/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 875us/step - accuracy: 0.7997 - loss: 0.4293 - val_accuracy: 0.8075 - val_loss: 0.4222
Epoch 5/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.8024 - loss: 0.4236 - val_accuracy: 0.8072 - val_loss: 0.4255
Epoch 6/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 855us/step - accuracy: 0.8014 - loss: 0.4266 - val_accuracy: 0.8059 - val_loss: 0.4238
Epoch 7/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 864us/step - accuracy: 0.8064 - loss: 0.4211 - val_accuracy: 0.8055 - val_loss: 0.4218
Epoch 8/10
1397/1397 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.8012 - loss: 0.42

In [ ]:
from mlflow.models.signature import infer_signature

# Generate the model signature based on the training data
signature = infer_signature(X_train, model.predict(X_train))

print("Model Signature:")
print(signature)

1747/1747 ━━━━━━━━━━━━━━━━━━━━ 1s 428us/step
Model Signature:
inputs: 
  [Tensor('float64', (-1, 11))]
outputs: 
  [Tensor('float32', (-1, 1))]
params: 
  None


Input Example:
[[ 0.70400433  0.18859525 -0.34915267  0.68488166  4.20123779  0.57789035
  -0.85620974 -0.19822058 -0.60802359 -0.51070781 -1.02779073]
 [ 0.71638988  0.29579343  0.71698978 -0.97175366  0.87898701  0.57789035
   1.16793813  0.60582979 -0.60802359  1.95806677 -1.02779073]
 [-0.16298391 -0.77618838  0.71698978  1.21192243 -0.78213838  0.57789035
  -0.85620974  0.07870098 -0.60802359 -0.51070781  0.97296071]
 [-0.95565887  0.29579343 -0.34915267  0.61686682 -0.78213838  0.57789035
   1.16793813 -0.15208642  1.64467302 -0.51070781  0.97296071]
 [-1.37676745  0.83178433 -0.34915267 -0.97175366  0.87898701  0.57789035
   1.16793813 -0.18602756 -0.60802359 -0.51070781  0.97296071]]


In [19]:
import numpy as np

input_example = np.array(X_test[:5])

In [20]:
input_example

array([[ 0.70400433,  0.18859525, -0.34915267,  0.68488166,  4.20123779,
         0.57789035, -0.85620974, -0.19822058, -0.60802359, -0.51070781,
        -1.02779073],
       [ 0.71638988,  0.29579343,  0.71698978, -0.97175366,  0.87898701,
         0.57789035,  1.16793813,  0.60582979, -0.60802359,  1.95806677,
        -1.02779073],
       [-0.16298391, -0.77618838,  0.71698978,  1.21192243, -0.78213838,
         0.57789035, -0.85620974,  0.07870098, -0.60802359, -0.51070781,
         0.97296071],
       [-0.95565887,  0.29579343, -0.34915267,  0.61686682, -0.78213838,
         0.57789035,  1.16793813, -0.15208642,  1.64467302, -0.51070781,
         0.97296071],
       [-1.37676745,  0.83178433, -0.34915267, -0.97175366,  0.87898701,
         0.57789035,  1.16793813, -0.18602756, -0.60802359, -0.51070781,
         0.97296071]])

In [9]:
with mlflow.start_run():
    mlflow.keras.log_model(model, "keras_model", registered_model_name="ChurnPredictionModel")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_param("epochs", 10)
    mlflow.log_param("batch_size", 32)

2025/05/03 17:27:14 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/05/03 17:27:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ChurnPredictionModel'.
Created version '1' of model 'ChurnPredictionModel'.


In [10]:
# Load the model for inference
loaded_model = mlflow.keras.load_model("models:/ChurnPredictionModel/1")
# Make predictions
predictions = loaded_model.predict(X_test[:5])
predictions = (predictions > 0.5).astype(int)  # Convert probabilities to binary predictions
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[[1]
 [0]
 [0]
 [1]
 [0]]


In [11]:
sampled_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
65063,652,32.0,8,167664.83,2,1.0,1.0,125435.47,0,False,False,False
135598,687,36.0,4,0.00,3,1.0,1.0,132460.29,1,False,False,True
138368,690,53.0,0,127900.97,2,1.0,0.0,67830.90,1,True,False,False
99201,659,43.0,1,0.00,1,1.0,0.0,121084.27,1,False,False,True
61953,512,38.0,1,160541.00,1,0.0,0.0,164038.07,1,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
31115,619,26.0,8,0.00,2,1.0,1.0,95937.98,0,False,False,True
29976,534,39.0,3,0.00,1,0.0,0.0,25843.70,1,False,False,True
115411,516,31.0,5,0.00,1,1.0,1.0,173783.38,0,False,True,True
4050,715,40.0,8,0.00,1,1.0,1.0,50560.37,1,False,False,True


In [30]:
train_data = pd.DataFrame(X_train, columns=X.columns)
train_data["probability"] = loaded_model.predict(X_train)
train_data["prediction"] = (train_data["probability"] > 0.5).astype(int)
train_data["actual"] = y_train.values

1747/1747 ━━━━━━━━━━━━━━━━━━━━ 1s 444us/step


In [27]:
features = sampled_df.drop(["Exited"], axis=1).columns

In [32]:
train_data.to_csv("train_data.csv", index=False)